# Market Data Analysis by FinMAS

This notebook can be used to run an analysis by Market Data crew instead of using the UI / panel app.

In [ ]:
# Convenience function to change the working directory to the root of the project if necessary
from pathlib import Path
import os

if not Path("finmas").exists():
    os.chdir("..")
print(f"Current working directory: {Path.cwd()}")

In [ ]:
%%capture
import time

from IPython.display import Markdown, display

from finmas.crews import MarketDataCrew
from finmas.crews.utils import CrewRunMetrics, save_crew_output, get_yaml_config_as_markdown
from finmas.utils.common import format_time_spent

To be able to run this notebook make sure the necessary API_KEY for the LLM is set in the `.env` file
or set in the environment variables manually

| provider | API_KEY |
| --- | --- |
| groq | GROQ_API_KEY |
| openai | OPENAI_API_KEY |

## Configuration

List of Groq models: https://console.groq.com/settings/limits

In [ ]:
TICKER = "META"

# LLM_PROVIDER = "groq"
# LLM_MODEL = "llama3-8b-8192"

LLM_PROVIDER = "openai"
LLM_MODEL = "gpt-4o"
# LLM_MODEL = "gpt-4o-mini"

# LLM configuration
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 1024

CREW_NAME = "market_data"

## Crew Configuration

In [ ]:
config_path = Path.cwd() / "finmas" / "crews" / CREW_NAME / "config"

### Agents

In [ ]:
display(Markdown(get_yaml_config_as_markdown(config_path, "agents")))

### Tasks Definition

In [ ]:
display(Markdown(get_yaml_config_as_markdown(config_path, "tasks")))

## Fundamentals Tool

To demonstrate the tool that the fundamental analyst agent uses, we show the following example output.

This tool extracts the latest fundamental data using Alpha Vantage for income statement and balance sheet, and Yahoo Finance for price data.  
The tool condenses the data into 2 tables formatted as Markdown to make it easier for the LLM agent to interpret the data.

In [ ]:
from finmas.data.market.fundamentals import StockFundamentalsTool

fundamentals_tool = StockFundamentalsTool()

display(Markdown(fundamentals_tool.run(TICKER)))

## Technical Analysis Tool

To demonstrate the tool that the technical analyst agent uses, we show the following example output.\
The tool extracts the latest price data using Yahoo Finance and resamples the data to a weekly timeframe.\
The technical indicators 20 week moving average, 50 week moving average are used for indicating the trend of the stock.\
The momentum indicator RSI14 and the volatility indicator Bollinger Band percentage are used\
to indicate the strength of the momentum and the volatility of the stock.

In [ ]:
from finmas.data.market.technical_analysis import TechnicalAnalysisTool

ta_tool = TechnicalAnalysisTool()

display(Markdown(ta_tool.run(TICKER)))

## Create Crew

This section creates the crew with the configuration of agents and tasks defined above.
The crew will use the output from the tools shown above to make a decision on the stock.

In [ ]:
%%capture
start = time.time()
crew = MarketDataCrew(
    ticker=TICKER,
    llm_provider=LLM_PROVIDER,
    llm_model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    max_tokens=LLM_MAX_TOKENS,
)

## Run the analysis

In [ ]:
%%capture
inputs = {"ticker": TICKER}  # "form": filing.form}
output = crew.crew().kickoff(inputs=inputs)
time_spent = time.time() - start
display(f"Time spent: {format_time_spent(time_spent)}")

## Save the results

In [ ]:
crew_run_metrics = CrewRunMetrics(
    config=crew.config, token_usage=output.token_usage, time_spent=time_spent
)
file_path = save_crew_output(crew_run_metrics, output.raw)
display(f"Output saved to: {file_path}")

## Display the results

In [ ]:
crew_output = file_path.read_text(encoding="utf-8")
display(Markdown(crew_output))